# Basic Examples of Metrics with Prometheus Operator

## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 

## Setup Seldon Core

Install Seldon Core as described in [docs](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html).

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:
```bash
kubectl port-forward -n istio-system svc/istio-ingressgateway 8003:80
```

In [1]:
%%bash
kubectl create namespace seldon || echo "Seldon namespace already exists"
kubectl config set-context $(kubectl config current-context) --namespace=seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


Seldon namespace already exists
Context "kind-ansible" modified.


## Install Prometheus Operator

In [2]:
%%bash
kubectl create namespace seldon-system

# Note: we set prometheus.scrapeInterval=1s for CI tests reliability here
helm upgrade --install prometheus kube-prometheus \
    --version 6.8.3 \
    --set fullnameOverride=seldon-monitoring \
    --set prometheus.scrapeInterval=1s \
    --namespace seldon-system \
    --repo https://charts.bitnami.com/bitnami

Error from server (AlreadyExists): namespaces "seldon-system" already exists


Release "prometheus" has been upgraded. Happy Helming!
NAME: prometheus
LAST DEPLOYED: Mon Apr 11 10:15:21 2022
NAMESPACE: seldon-system
STATUS: deployed
REVISION: 2
TEST SUITE: None
NOTES:
CHART NAME: kube-prometheus
CHART VERSION: 6.8.3
APP VERSION: 0.55.1

** Please be patient while the chart is being deployed **

Watch the Prometheus Operator Deployment status using the command:

    kubectl get deploy -w --namespace seldon-system -l app.kubernetes.io/name=kube-prometheus-operator,app.kubernetes.io/instance=prometheus

Watch the Prometheus StatefulSet status using the command:

    kubectl get sts -w --namespace seldon-system -l app.kubernetes.io/name=kube-prometheus-prometheus,app.kubernetes.io/instance=prometheus

Prometheus can be accessed via port "9090" on the following DNS name from within your cluster:

    seldon-monitoring-prometheus.seldon-system.svc.cluster.local

To access Prometheus from outside the cluster execute the following commands:

    echo "Prometheus URL: htt

In [3]:
%%bash
# Extra sleep as statefulset is not always present right away
sleep 5 
kubectl rollout status -n seldon-system deployment/seldon-monitoring-operator
kubectl rollout status -n seldon-system deployment/prometheus-kube-state-metrics
kubectl rollout status -n seldon-system statefulsets/prometheus-seldon-monitoring-prometheus

deployment "seldon-monitoring-operator" successfully rolled out
deployment "prometheus-kube-state-metrics" successfully rolled out
statefulset rolling update complete 1 pods at revision prometheus-seldon-monitoring-prometheus-c59cdd94d...


In [4]:
%%bash
cat <<EOF | kubectl apply -f -
apiVersion: monitoring.coreos.com/v1
kind: PodMonitor
metadata:
  name: seldon-podmonitor
spec:
  selector:
    matchLabels:
      app.kubernetes.io/managed-by: seldon-core
  podMetricsEndpoints:
    - port: metrics
      path: /prometheus
  namespaceSelector:
    any: true
EOF

podmonitor.monitoring.coreos.com/seldon-podmonitor unchanged


## Deploy Example Model

In [5]:
%%writefile echo-sdep.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: echo
  namespace: seldon
spec:
  predictors:
  - name: default
    replicas: 1
    graph:
      name: classifier
      type: MODEL
    componentSpecs:
    - spec:
        containers:
        - image: seldonio/echo-model:1.14.0-dev
          name: classifier

Overwriting echo-sdep.yaml


In [6]:
!kubectl apply -f echo-sdep.yaml

seldondeployment.machinelearning.seldon.io/echo unchanged


In [7]:
%%bash
deployment=$(kubectl get deploy -l seldon-deployment-id=echo -o jsonpath='{.items[0].metadata.name}')
kubectl rollout status deploy/${deployment}

deployment "echo-default-0-classifier" successfully rolled out


## Sent series of REST requests

In [8]:
%%bash

# Wait for the model to become fully ready
echo "Waiting 5s for model to fully ready"
sleep 5

# Send 20 requests to REST endpoint
for i in `seq 1 10`; do sleep 0.1 && \
   curl -s -H "Content-Type: application/json" \
   -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   http://localhost:8003/seldon/seldon/echo/api/v1.0/predictions > /dev/null ; \
done

# Give time for metrics to get collected by Prometheus
echo "Waiting 10s for Prometheus to scrape metrics"
sleep 10

Waiting 5s for model to fully ready
Waiting 10s for Prometheus to scrape metrics


## Check Metrics (REST)

In [9]:
import json

In [10]:
%%writefile get-metrics.sh
QUERY='query=seldon_api_executor_client_requests_seconds_count{deployment_name=~"echo",namespace=~"seldon",method=~"post"}'
QUERY_URL=http://seldon-monitoring-prometheus.seldon-system.svc.cluster.local:9090/api/v1/query

kubectl run --quiet=true -it --rm curlmetrics-$(date +%s) --image=radial/busyboxplus:curl --restart=Never -- \
    curl --data-urlencode ${QUERY} ${QUERY_URL}

Overwriting get-metrics.sh


In [11]:
metrics = ! bash get-metrics.sh
metrics = json.loads(metrics[0])

In [12]:
metrics

{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'__name__': 'seldon_api_executor_client_requests_seconds_count',
     'code': '200',
     'container': 'seldon-container-engine',
     'deployment_name': 'echo',
     'endpoint': 'metrics',
     'instance': '10.244.0.16:8000',
     'job': 'seldon/seldon-podmonitor',
     'method': 'post',
     'model_image': 'seldonio/echo-model',
     'model_name': 'classifier',
     'model_version': '1.14.0-dev',
     'namespace': 'seldon',
     'pod': 'echo-default-0-classifier-699db857fb-msp7v',
     'predictor_name': 'default',
     'service': '/predict'},
    'value': [1649668548.664, '10']}]}}

In [13]:
counter = int(metrics["data"]["result"][0]["value"][1])
assert counter == 10, f"expected 10 requests, got {counter}"

## Send series GRPC requests

In [14]:
%%bash
cd ../../../executor/proto && for i in `seq 1 10`; do sleep 0.1 && \
    grpcurl -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
    -rpc-header seldon:echo -rpc-header namespace:seldon \
    -plaintext -proto ./prediction.proto \
     0.0.0.0:8003 seldon.protos.Seldon/Predict > /dev/null ; \
done

# Give time for metrics to get collected by Prometheus
echo "Waiting 10s for Prometheus to scrape metrics"
sleep 10

Waiting 10s for Prometheus to scrape metrics


## Check metrics (GRPC)

In [15]:
%%writefile get-metrics.sh
QUERY='query=seldon_api_executor_client_requests_seconds_count{deployment_name=~"echo",namespace=~"seldon",method=~"unary"}'
QUERY_URL=http://seldon-monitoring-prometheus.seldon-system.svc.cluster.local:9090/api/v1/query

kubectl run --quiet=true -it --rm curlmetrics-$(date +%s) --image=radial/busyboxplus:curl --restart=Never -- \
    curl --data-urlencode ${QUERY} ${QUERY_URL}

Overwriting get-metrics.sh


In [16]:
metrics = ! bash get-metrics.sh
metrics = json.loads(metrics[0])

In [17]:
counter = int(metrics["data"]["result"][0]["value"][1])
assert counter == 10, f"expected 10 requests, got {counter}"

## Check Custom Metrics

This model defines a few custom metrics in its `.py` class definition:
```Python
    def metrics(self):
        print("metrics called")
        return [
            # a counter which will increase by the given value
            {"type": "COUNTER", "key": "mycounter", "value": 1},

            # a gauge which will be set to given value
            {"type": "GAUGE", "key": "mygauge", "value": 100},

            # a timer (in msecs) which  will be aggregated into HISTOGRAM
            {"type": "TIMER", "key": "mytimer", "value": 20.2},
        ]
```      

We will be checking value of `mygaguge` metrics.

In [18]:
%%writefile get-metrics.sh
QUERY='query=mygauge{deployment_name=~"echo",namespace=~"seldon"}'
QUERY_URL=http://seldon-monitoring-prometheus.seldon-system.svc.cluster.local:9090/api/v1/query

kubectl run --quiet=true -it --rm curlmetrics-$(date +%s) --image=radial/busyboxplus:curl --restart=Never -- \
    curl --data-urlencode ${QUERY} ${QUERY_URL}

Overwriting get-metrics.sh


In [19]:
metrics = ! bash get-metrics.sh
metrics = json.loads(metrics[0])

In [20]:
metrics

{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'__name__': 'mygauge',
     'container': 'classifier',
     'deployment_name': 'echo',
     'endpoint': 'metrics',
     'image_name': 'seldonio/echo-model',
     'image_version': '1.14.0-dev',
     'instance': '10.244.0.16:6000',
     'job': 'seldon/seldon-podmonitor',
     'method': 'predict',
     'model_image': 'seldonio/echo-model',
     'model_name': 'classifier',
     'model_version': '1.14.0-dev',
     'namespace': 'seldon',
     'pod': 'echo-default-0-classifier-699db857fb-msp7v',
     'predictor_name': 'default',
     'predictor_version': 'default',
     'seldon_deployment_name': 'echo',
     'worker_id': '45'},
    'value': [1649668563.21, '100']},
   {'metric': {'__name__': 'mygauge',
     'container': 'classifier',
     'deployment_name': 'echo',
     'endpoint': 'metrics',
     'image_name': 'seldonio/echo-model',
     'image_version': '1.14.0-dev',
     'instance': '10.244.0.16:6000',
     'j

In [21]:
gauge = int(metrics["data"]["result"][0]["value"][1])
assert gauge == 100, f"expected 100 on guage, got {gauge}"

## Cleanup

In [22]:
!kubectl delete sdep -n seldon echo
!helm uninstall -n seldon-system prometheus

seldondeployment.machinelearning.seldon.io "echo" deleted
release "prometheus" uninstalled
